In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several "helpful packages to load
import warnings
warnings.filterwarnings('ignore')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

sns.color_palette("YlGnBu", 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

## 1. Reading and understanding the data

In [8]:
# Reading the training data
data = pd.read_csv('./train.csv')
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [9]:
data.shape

(1460, 81)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

## 2.Exploratory Data Analysis (EDA)

### 2.1. Data missing value check
- Percentage of data missing in any of the columns

In [11]:
columns_with_null = round(data.isnull().sum() / len(data)  * 100, 2)
columns_with_null[columns_with_null > 0]

LotFrontage     17.74
Alley           93.77
MasVnrType       0.55
MasVnrArea       0.55
BsmtQual         2.53
BsmtCond         2.53
BsmtExposure     2.60
BsmtFinType1     2.53
BsmtFinType2     2.60
Electrical       0.07
FireplaceQu     47.26
GarageType       5.55
GarageYrBlt      5.55
GarageFinish     5.55
GarageQual       5.55
GarageCond       5.55
PoolQC          99.52
Fence           80.75
MiscFeature     96.30
dtype: float64

In [12]:
# Checking the columns with higher percentage of missing values say 50%
null_more_half = columns_with_null[columns_with_null > 50].index
null_more_half

Index(['Alley', 'PoolQC', 'Fence', 'MiscFeature'], dtype='object')

#### Columns - `Id`, `Alley`, `PoolQC`, `Fence`, `MiscFeature` have highest percentage of missing data, here we can drop them

In [13]:
data = data.drop(null_more_half,  axis=1)
data = data.drop('Id',  axis=1)

In [14]:
# Checking the rest of columns that have null values
columns_with_null = round(data.isnull().sum() / len(data)  * 100, 2)

columns_with_null[columns_with_null > 0]

LotFrontage     17.74
MasVnrType       0.55
MasVnrArea       0.55
BsmtQual         2.53
BsmtCond         2.53
BsmtExposure     2.60
BsmtFinType1     2.53
BsmtFinType2     2.60
Electrical       0.07
FireplaceQu     47.26
GarageType       5.55
GarageYrBlt      5.55
GarageFinish     5.55
GarageQual       5.55
GarageCond       5.55
dtype: float64

#### Dropping column `FireplaceQu`

Column `FireplaceQu` has approx 47% data missing in it

In [ ]:
data = data.drop('FireplaceQu', axis=1)

In [ ]:
columns_with_null = round(data.isnull().sum() / len(data)  * 100, 2)

columns_with_null[columns_with_null > 0]

### 2.2. Filling missing values

We will be filling the missing values of continuous variables with the `mean` and the categorical variables with `mode` of the columns. Below is the list of the categorical and continuos variables.

  - LotFrontage
  - MasVnrArea
  - GarageYrBlt
  - MasVnrType
  - BsmtQual
  - BsmtCond
  - BsmtExposure
  - BsmtFinType1
  - BsmtFinType2
  - Electrical
  - GarageType
  - GarageFinish
  - GarageQual
  - GarageCond

In [ ]:
# Filling missing values of column LotFrontage with the mean of the columns
data['LotFrontage'] = data.LotFrontage.fillna(data.LotFrontage.mean())

In [ ]:
# Replacing the null value with the mean of the columns
data['MasVnrArea'] = data.MasVnrArea.fillna(data.MasVnrArea.mean())
data['GarageYrBlt'] = data.GarageYrBlt.fillna(data.GarageYrBlt.mean())

In [ ]:
# Filling categorical columns with the mode values
columns_with_null = round(data.isnull().sum() / len(data)  * 100, 2)
cat_null_cols = columns_with_null[columns_with_null > 0].index
cat_null_cols = data[cat_null_cols].select_dtypes('object').columns
data[cat_null_cols] = data[cat_null_cols].fillna(data[cat_null_cols].mode().iloc[0])
data[cat_null_cols].isnull().sum()

In [ ]:
# Final null value check to check if there are any columns with null values
data.columns[data.isnull().sum() / len(data) > 0]

### 2.3 Outlier detection

We will be capping the outliers for each detected column that has outliers

In [ ]:
data.describe()

#### Columns that seems to have outliers

- LotFrontage
- LotArea
- MasVnrArea
- BsmtFinSF1
- BsmtUnfSF
- TotalBsmtSF
- 1stFlrSF
- 2ndFlrSF
- GrLivArea
- GarageArea
- WoodDeckSF
- OpenPorchSF
- EnclosedPorch
- 3SsnPorch
- ScreenPorch
- PoolArea
- MiscVal
- SalePrice

In [ ]:
def plot_outliers(data, cols):
    figure = 1
    row = 1
    plt.figure(figsize=(15, 15))
    for i in range(len(cols)):
        fig = str(9) + "3" + str(figure)
        plt.subplot(int(fig))
        sns.boxplot(data[cols[i]])
        if figure == 3:
            plt.figure(figsize=(15, 15))
            figure = 1
            row += 1
        else:
            figure += 1

    plt.show()
    
def plot_dist(data, cols):
    figure = 1
    row = 1
    plt.figure(figsize=(15, 15))
    for i in range(len(cols)):
        try:
            fig = str(9) + "3" + str(figure)
            plt.subplot(int(fig))
            sns.distplot(data[cols[i]])
            if figure == 3:
                plt.figure(figsize=(15, 15))
                figure = 1
                row += 1
            else:
                figure += 1
        except:
            print("Error plotting column ", cols[i])

    plt.show()

In [ ]:
outlier_columns = ['LotFrontage' ,'LotArea', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', 'GarageArea', 'SalePrice']

percentiles = data['LotFrontage'].quantile([0.05,0.95]).values
data['LotFrontage'][data['LotFrontage'] <= percentiles[0]] = percentiles[0]
data['LotFrontage'][data['LotFrontage'] >= percentiles[1]] = percentiles[1]

In [ ]:
percentiles = data['LotArea'].quantile([0.05, 0.95]).values
data['LotArea'][data['LotArea'] <= percentiles[0]] = percentiles[0]
data['LotArea'][data['LotArea'] >= percentiles[1]] = percentiles[1]

In [ ]:
max_percentile = data['BsmtUnfSF'].quantile(0.97)
data['BsmtUnfSF'][data['BsmtUnfSF'] >= max_percentile] = max_percentile

In [ ]:
percentiles = data['TotalBsmtSF'].quantile([0.05, 0.98]).values
data['TotalBsmtSF'][data['TotalBsmtSF'] <= percentiles[0]] = percentiles[0]
data['TotalBsmtSF'][data['TotalBsmtSF'] >= percentiles[1]] = percentiles[1]

In [ ]:
max_percentile = data['1stFlrSF'].quantile(0.98)
data['1stFlrSF'][data['1stFlrSF'] >= max_percentile] = max_percentile

In [ ]:
max_percentile = data['GarageArea'].quantile(0.98)
data['GarageArea'][data['GarageArea'] >= max_percentile] = max_percentile

In [ ]:
max_percentile = max_percentile = data['SalePrice'].quantile(0.95)
data['SalePrice'][data['SalePrice'] >= max_percentile] = max_percentile

### After capping the outliers

In [ ]:
plot_outliers(data, outlier_columns)

In [ ]:
plot_dist(data, outlier_columns)

In [ ]:
data.shape

## Data Preperation

In this step, we will prepare the data to fed into the learning algorithm.

### Creating dummy columns

We will not covert all of the categorical columns to dummy variables

In [ ]:
data.info()

In [ ]:
cat_columns = data.select_dtypes('object').columns
cat_columns

In [ ]:
cat_dummies = pd.get_dummies(data[cat_columns], drop_first=True)
cat_dummies.head()

In [ ]:
data = data.drop(cat_columns, axis=1)
data.head()

In [ ]:
data = pd.concat([data, cat_dummies], axis=1)
data.head()

In [ ]:
def get_corr_features(df, target=None, thres=0.8):
    corr_df = df.copy()
    if target is not None:
        corr_df = df.drop(columns = target)
    
    corr = corr_df.corr()
    
    cols = corr.columns
    corr_feature = set()
    
    for i in range(len(cols)):
        for j in range(i):
            if abs(corr.iloc[i, j]) >= thres:
                corr_feature.add(cols[i])
    
    return corr_feature

In [ ]:
def calculate_vif(x, target=None, thresh=5):
    # Create a dataframe that will contain the names of all the feature variables and their respective VIFs
    vif_x = x.copy()
    
    if target is not None:
        vif_x = vif_x.drop(columns=target)
    
    features = vif_x.columns
    feature_indexs = np.arange(len(features))
    
    keep_dropping = True
    
    print('Calculating VIF')
    
    while keep_dropping:
        
        keep_dropping = False
        
        current_df_values = vif_x[features[feature_indexs]].values
    
        vifs = [variance_inflation_factor(current_df_values, i) for i in np.arange(current_df_values.shape[1])]
    
        max_vif = max(vifs)
        
        if max_vif > thresh:
            keep_dropping = True
            max_vif_index = vifs.index(max_vif)
            feature_indexs = np.delete(feature_indexs, max_vif_index)
        print('Feature left', len(feature_indexs))
    
    print('Returning', len(feature_indexs))
    return feature_indexs

In [ ]:
corr_feature = get_corr_features(data, 'SalePrice')
if len(corr_feature) > 0:
    print('Dropping highly corr features')
    data = data.drop(columns = corr_feature)

In [ ]:
feature_idx_to_keep = calculate_vif(data, 'SalePrice')

if len(feature_idx_to_keep) > 0:
    print('Dropping the variables with high VIF')
    data = data[data.columns[feature_idx_to_keep]]

In [ ]:
data.shape

In [ ]:
train, test = train_test_split(data, train_size=0.8, random_state=100)

train.shape, test.shape

*Creating X_train, y_train, X_test, y_test*

In [ ]:
X_train = train.drop('SalePrice', axis=1)
y_train = train['SalePrice']

In [ ]:
y_train.head()

In [ ]:
# Create the parameter grid based on the results of random search 
params = {
    'max_depth': [25, 26, 27, 28, 29],
    'min_samples_split': [2, 4, 6, 8, 10],
    'min_samples_leaf': [5, 10, 20, 50, 100],
    'max_features': [27, 30, 31, 32, 33],
    'n_estimators': [100, 200, 250, 300]
}

In [ ]:
classifier_rf = RandomForestRegressor(random_state=42, n_jobs=-1)
# Instantiate the grid search model
grid_search = GridSearchCV(estimator=classifier_rf, param_grid=params, 
                          cv=4, n_jobs=-1, verbose=1, scoring = "r2", refit=True)

In [ ]:
grid_search.fit(X_train,y_train)

In [ ]:
round(grid_search.best_score_, 2), grid_search.best_params_

In [ ]:
grid_search.

In [ ]:
X_train_sm = sm.add_constant(X_train)
ols = sm.OLS(y_train, X_train_sm).fit()

print(ols.summary())

In [ ]:
# Let's visualise the data with a scatter plot and the fitted regression line
plt.scatter(X_train_sm.iloc[:, 1], y_train)
plt.plot(X_train_sm.iloc[:, 1], 0.127 + 0.462*X_train_sm.iloc[:, 1], 'r')
plt.show()

## Creating Pipeline

In [ ]:
scaler = MinMaxScaler()
estimator = LinearRegression()
selector_rfe = RFE(estimator, n_features_to_select=2)

pipeline = Pipeline([('scaler', scaler),
                     ('estimator', selector_rfe)])
pipeline

Creating GridSearchSpace

**Step 1:** Creating a cross-validation scheme and hyperparameter grid

In [ ]:
kfold = KFold(n_splits=4, shuffle= True, random_state=100)
hyper_params = [{'estimator__n_features_to_select': range(2, 70)}]

**Step 2:** Perform GridSearch

In [ ]:
model_cv = GridSearchCV(estimator=pipeline,
                       param_grid=hyper_params,
                       cv=kfold,
                       scoring='r2',
                       verbose=1,
                       return_train_score=True,
                       n_jobs=-1,
                       refit=True)

In [ ]:
model_cv.fit(X_train, y_train)

In [ ]:
# cv results
cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results.head()

In [ ]:
round(grid_search.best_score_, 2), grid_search.best_params_

In [ ]:
# plotting cv results
plt.figure(figsize=(16,6))

plt.plot(cv_results["param_estimator__n_features_to_select"], cv_results["mean_test_score"])
plt.plot(cv_results["param_estimator__n_features_to_select"], cv_results["mean_train_score"])
plt.xlabel('number of features')
plt.ylabel('r-squared')
# plt.yscale('log')
plt.title("Optimal Number of Features")
plt.legend(['test score', 'train score'], loc='upper left')

In [ ]:
estimator = model_cv.best_estimator_
estimator

In [ ]:
scaler = estimator.steps[0][1]
rfe = estimator.steps[1][1]

In [ ]:
list(zip(X_train.columns, rfe.support_, rfe.ranking_))

In [ ]:
def error_terms(y_train, y_train_pred):
    plt.figure()
    sns.distplot((y_train - y_train_pred))
    plt.title('Error terms', fontsize=20)
    plt.xlabel('Errors', fontsize=16)
    plt.show()

In [ ]:
y_train_pred = grid_search.predict(X_train)
y_train_pred

In [ ]:
print('MSE of train pred', round(mean_squared_error(y_train, y_train_pred), 2))
print('R2 Score of train pred', round(r2_score(y_train, y_train_pred), 2))

In [ ]:
error_terms(y_train, y_train_pred)

In [ ]:
plt.scatter((y_train - y_train_pred), y_train_pred)
plt.show()

In [ ]:
y_test = test['SalePrice']
x_test = test.drop(columns = 'SalePrice')

x_test.head()

In [ ]:
x_test_1 = x_test[X_train_m1.columns.values]
x_test_1.head()

x_test_1.shape

In [ ]:
x_test_1_sm = sm.add_constant(x_test_1)
y_test_pred = ols_m1.predict(x_test_1_sm)
y_test_pred

In [ ]:
print('MSE of test pred', round(mean_squared_error(y_test, y_test_pred), 2))
print('R2 Score', round(r2_score(y_test, y_test_pred), 2))